In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key=os.getenv('GOOGLE_API_KEY')
if not api_key:
    raise ('Not key')

In [3]:
MAX_PAGE=2
URL='https://cafef.vn/tai-chinh-ngan-hang.chn'
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
    'Accept-Language': 'vi-VN,vi;q=0.9,en-US;q=0.8,en;q=0.7'
}
response=requests.get(URL,headers=HEADERS,timeout=10)

In [22]:
if response.status_code == 200:
    soup=BeautifulSoup(response.content,'html.parser')
# soup

In [5]:
hiden_input=soup.find('input',{'id':'hdZoneId'})
hiden_input

<input id="hdZoneId" name="hdZoneId" type="hidden" value="18834">
<input id="hdZoneUrl" name="hdZoneUrl" type="hidden" value="tai-chinh-ngan-hang"/>
<input id="hdParentUrl" name="hdParentUrl" type="hidden" value=""/>
<input id="hdZoneParentUrl" name="hdZoneParentUrl" type="hidden" value=""/>
</input>

In [6]:
zoneId=hiden_input.get('value')
zoneId

'18834'

In [7]:
all_link=[]
base_api='https://cafef.vn/timelinelist/{}/{}.chn'
for page in range(1, MAX_PAGE+1):
    api_url=base_api.format(zoneId,page)

    r=requests.get(api_url,headers=HEADERS,timeout=10)
    if r.status_code==200:
        api_soup=BeautifulSoup(r.content,'html.parser')
        links_found = 0

        for a_tag in api_soup.find_all('a',href=True):
            href=a_tag['href']
            if href.endswith('.chn') and len(href)>25:
                full_link = "https://cafef.vn" + href if not href.startswith('http') else href
                if full_link not in all_link:
                    all_link.append(full_link)
time.sleep(1)
print(f'{len(all_link)}')
print(f'{all_link[:5]}')

16
['https://cafef.vn/ngan-hang-niem-yet-lai-suat-tiet-kiem-cao-hang-dau-bat-ngo-dieu-chinh-manh-ky-han-dai-gui-6-thang-lai-cao-hon-36-thang-188260119111803566.chn', 'https://cafef.vn/sacombank-tang-manh-lai-suat-tiet-kiem-188260119103353382.chn', 'https://cafef.vn/trua-19-1-gia-vang-nhan-vang-mieng-tang-hon-2-trieu-dong-cham-moc-165-trieu-dong-luong-188260119082009124.chn', 'https://cafef.vn/cap-nhat-thi-truong-tien-te-ty-gia-usd-tu-do-giam-sau-ngan-hang-nha-nuoc-hut-ve-gan-30000-ty-dong-trong-tuan-qua-188260119102150209.chn', 'https://cafef.vn/woori-bank-trien-khai-uu-dai-ra-mat-the-tin-dung-cao-cap-voi-loat-qua-tang-trai-nghiem-188260117124055631.chn']


In [8]:
final_data=[]
for i,url in enumerate(all_link[:5]):
    r=requests.get(url,headers=HEADERS,timeout=10)
    soup=BeautifulSoup(r.content,'html.parser')

    title=soup.find('h1',class_='title')
    title_text=title.text.strip() if title else''

    content_div=soup.find('div',class_='detail-content')
    if not content_div:
        content_div=soup.find('div',class_='contentdetail')
    content_text=''
    if content_div:
        for script in content_div(["script", "style", "iframe"]):
            script.decompose()
        paragraphs = [p.text.strip() for p in content_div.find_all('p') if p.text.strip()]
        content_text = "\n".join(paragraphs)
    if title_text and len(content_text) > 50:
        final_data.append({
            'url':url,
            'title':title_text,
            'content':content_text
        })
    time.sleep(1)
df = pd.DataFrame(final_data)

In [9]:
df.head(5)

,url,title,content
0,https://cafef.vn/ngan-hang-niem-yet-lai-suat-t...,Ngân hàng niêm yết lãi suất tiết kiệm cao hàng...,Ngân hàng TMCP Thịnh vượng và Phát triển (PGBa...
1,https://cafef.vn/sacombank-tang-manh-lai-suat-...,Sacombank tăng mạnh lãi suất tiết kiệm,"Đối với gửi tiết kiệm tại quầy, lãi suất kỳ hạ..."
2,https://cafef.vn/trua-19-1-gia-vang-nhan-vang-...,"Trưa 19/1: Giá vàng nhẫn, vàng miếng tăng hơn ...","Tại thời điểm khảo sát, giá vàng miếng tại các..."
3,https://cafef.vn/cap-nhat-thi-truong-tien-te-t...,Cập nhật thị trường tiền tệ: Tỷ giá USD tự do ...,Nghiệp vụ thị trường mở: Trên kênh cho vay cầm...
4,https://cafef.vn/woori-bank-trien-khai-uu-dai-...,Woori Bank triển khai ưu đãi ra mắt thẻ tín dụ...,"Theo đó, khách hàng mở thẻ Woori VV Premium và..."


In [10]:
import re

def clean_code(text):
    text=re.sub(r'[^\w\s.,%()-]',' ',text)
    text=re.sub(r'(Theo|Nguồn):.*$',' ',text)
    text=re.sub(r'\s+',' ',text).strip()
    return text

df['content']=df['content'].apply(clean_code)
df['content'].iloc[0]

'Ngân hàng TMCP Thịnh vượng và Phát triển (PGBank) vừa bất ngờ điều chỉnh giảm mạnh lãi suất tiền gửi ở các kỳ hạn dài, sau giai đoạn tăng nóng liên tiếp. Trước đó, trong tháng 12 2025, ngân hàng này từng có tới 3 lần điều chỉnh tăng lãi suất liên tiếp lên mức cao nhất hệ thống, với mức tăng tại nhiều kỳ hạn lên tới 2,2% năm. Theo biểu lãi suất tiền gửi VND dành cho khách hàng cá nhân lĩnh lãi cuối kỳ mới cập nhật, PGBank đồng loạt hạ lãi suất các kỳ hạn từ 18 36 tháng từ mức 7,3% năm xuống còn 6,8% năm, tương ứng mức giảm 0,5% năm. Trong khi đó, mặt bằng lãi suất các kỳ hạn còn lại được giữ nguyên tiền gửi 1 3 tuần tiếp tục ở mức 0,2% năm các kỳ hạn ngắn từ 1 5 tháng duy trì trần 4,75% năm theo quy định. Với nhóm kỳ hạn 6 - 13 tháng, lãi suất tiền gửi tại PGBank vẫn ở mức cao. Cụ thể, kỳ hạn 6 9 tháng được niêm yết 7,1% năm, còn kỳ hạn 12 13 tháng giữ nguyên 7,2% năm. Sau điều chỉnh, đây trở thành mức lãi suất cao nhất trong biểu lãi suất lĩnh lãi cuối kỳ của ngân hàng này đồng thời l

In [23]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

split=RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=100,
    separators=['\n\n','\n','.',' ','']
)
    
chunk=[]
for index,row in df.iterrows():
    doc = Document(
        page_content=row['content'],
        metadata={
            'source':row['url'],
            'title':row['title']
        }
    )
    text_slpit=split.split_documents([doc])
    chunk.extend(text_slpit)
# chunk

In [12]:
from langchain_huggingface import HuggingFaceEmbeddings

model='BAAI/bge-m3'
# model='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
embeddings=HuggingFaceEmbeddings(model_name=model)

In [13]:
from langchain_chroma.vectorstores import Chroma

chroma=Chroma.from_documents(
    chunk,
    embeddings,
    persist_directory='./chroma_db_economy'
)

retriever=chroma.as_retriever(search_kwargs={'k':10})

In [14]:
y='lãi suất tiền gửi tại PGBank'

context=retriever.invoke(y)

for doc in context:
    print(f'{doc.page_content}')
print(f'-------')
# context='\n\n'.join([doc.page_content for doc in context])
# context

. Với nhóm kỳ hạn 6 - 13 tháng, lãi suất tiền gửi tại PGBank vẫn ở mức cao. Cụ thể, kỳ hạn 6 9 tháng được niêm yết 7,1% năm, còn kỳ hạn 12 13 tháng giữ nguyên 7,2% năm. Sau điều chỉnh, đây trở thành mức lãi suất cao nhất trong biểu lãi suất lĩnh lãi cuối kỳ của ngân hàng này đồng thời lãi suất kỳ hạn 6 tháng còn cao hơn cả 36 tháng. Ở biểu lãi suất tiết kiệm trả lãi hằng tháng, PGBank cũng điều chỉnh giảm lãi suất các kỳ hạn dài
. Với nhóm kỳ hạn 6 - 13 tháng, lãi suất tiền gửi tại PGBank vẫn ở mức cao. Cụ thể, kỳ hạn 6 9 tháng được niêm yết 7,1% năm, còn kỳ hạn 12 13 tháng giữ nguyên 7,2% năm. Sau điều chỉnh, đây trở thành mức lãi suất cao nhất trong biểu lãi suất lĩnh lãi cuối kỳ của ngân hàng này đồng thời lãi suất kỳ hạn 6 tháng còn cao hơn cả 36 tháng. Ở biểu lãi suất tiết kiệm trả lãi hằng tháng, PGBank cũng điều chỉnh giảm lãi suất các kỳ hạn dài
. Với nhóm kỳ hạn 6 - 13 tháng, lãi suất tiền gửi tại PGBank vẫn ở mức cao. Cụ thể, kỳ hạn 6 9 tháng được niêm yết 7,1% năm, còn kỳ hạ

In [15]:
from langchain_chroma.vectorstores import Chroma

collection_metadata={'hnsw:space':'l2'}

chroma_dis=Chroma.from_documents(
    chunk,
    embeddings,
    persist_directory='./chroma_db_energy',
    collection_metadata=collection_metadata
)

In [16]:
y_dis='lãi suất tiền gửi tại PGBank'
retriever_dis=chroma_dis.as_retriever(search_kwargs={'k':10})
context_dis=retriever_dis.invoke(y_dis)
for doc in context_dis:
    print(f'{doc.page_content}')
# context_dis='\n\n'.join([doc.page_content for doc in context_dis])
# context_dis

. Với nhóm kỳ hạn 6 - 13 tháng, lãi suất tiền gửi tại PGBank vẫn ở mức cao. Cụ thể, kỳ hạn 6 9 tháng được niêm yết 7,1% năm, còn kỳ hạn 12 13 tháng giữ nguyên 7,2% năm. Sau điều chỉnh, đây trở thành mức lãi suất cao nhất trong biểu lãi suất lĩnh lãi cuối kỳ của ngân hàng này đồng thời lãi suất kỳ hạn 6 tháng còn cao hơn cả 36 tháng. Ở biểu lãi suất tiết kiệm trả lãi hằng tháng, PGBank cũng điều chỉnh giảm lãi suất các kỳ hạn dài
. Với nhóm kỳ hạn 6 - 13 tháng, lãi suất tiền gửi tại PGBank vẫn ở mức cao. Cụ thể, kỳ hạn 6 9 tháng được niêm yết 7,1% năm, còn kỳ hạn 12 13 tháng giữ nguyên 7,2% năm. Sau điều chỉnh, đây trở thành mức lãi suất cao nhất trong biểu lãi suất lĩnh lãi cuối kỳ của ngân hàng này đồng thời lãi suất kỳ hạn 6 tháng còn cao hơn cả 36 tháng. Ở biểu lãi suất tiết kiệm trả lãi hằng tháng, PGBank cũng điều chỉnh giảm lãi suất các kỳ hạn dài
. Với nhóm kỳ hạn 6 - 13 tháng, lãi suất tiền gửi tại PGBank vẫn ở mức cao. Cụ thể, kỳ hạn 6 9 tháng được niêm yết 7,1% năm, còn kỳ hạ

In [17]:
from sentence_transformers import CrossEncoder

model_cross='BAAI/bge-reranker-v2-m3'

rerank=CrossEncoder(
    model_cross,
    max_length=512,
    device='cpu'
)
pairs=[[y,doc.page_content]for doc in context]
score=rerank.predict(pairs)

pairs_dis=[[y_dis,doc.page_content] for doc in context_dis]
score_dis=rerank.predict(pairs_dis)

In [18]:
print(f'score {score}')
print(f'-------')
print(f'score_dis {score_dis}')

score [0.99855465 0.99855465 0.99855465 0.99855465 0.99842924 0.99842924
 0.99842924 0.99842924 0.99419916 0.99419916]
-------
score_dis [0.99855465 0.99855465 0.99855465 0.99855465 0.99842924 0.99842924
 0.99842924 0.99842924 0.99419916 0.99419916]


In [19]:
for i in score:
    energy=-float(i)
    print(f'score {energy}')
print(f'-------')

for i_dis in score_dis:
    energy_dis=-float(i_dis)
    print(f'score {energy_dis}')

score -0.9985546469688416
score -0.9985546469688416
score -0.9985546469688416
score -0.9985546469688416
score -0.9984292387962341
score -0.9984292387962341
score -0.9984292387962341
score -0.9984292387962341
score -0.9941991567611694
score -0.9941991567611694
-------
score -0.9985546469688416
score -0.9985546469688416
score -0.9985546469688416
score -0.9985546469688416
score -0.9984292387962341
score -0.9984292387962341
score -0.9984292387962341
score -0.9984292387962341
score -0.9941991567611694
score -0.9941991567611694


In [20]:
# from langchain_core.output_parsers import StrOutputParser
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_google_genai import ChatGoogleGenerativeAI

# model='gemini-2.5-flash'
# llm=ChatGoogleGenerativeAI(model=model,temperature=0)
# def answer(query):
#     context=retriever.invoke(query)
#     context='\n\n'.join([doc.page_content for doc in context])

#     template='''
#     `context{context}

#     question{question}
#     '''
#     prompt=ChatPromptTemplate.from_template(template)

#     chain= prompt | llm | StrOutputParser()

#     return chain.invoke({'context':context, 'question':query})

In [21]:
# y='Gia co phieu VCB'

# a=answer(y)

# a
# #